In [1]:
import torch
from torch import nn

In [6]:
class block(nn.Module):
    def __init__(self, in_channels, out_channels, identity_downsample=None):
        super().__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding="same")
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv1d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding="same")
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        self.identity = identity_downsample

    def forward(self, x):
        identity = x
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))

        if self.identity is not None:
            identity = self.identity(identity)

        x += identity
        x = self.relu(x)
        
        return x


In [11]:
class M34(nn.Module):
    def __init__(self,block, in_channels, layers):
        super().__init__()
        self.layers = layers

        self.conv1 = nn.Conv1d(1, 48, kernel_size=160, stride=4)
        self.bn1 = nn.BatchNorm1d(48)
        self.relu = nn.ReLU()

        self.maxpool = nn.MaxPool1d(4)

        self.layer1 = self._make_layer(block, layers[0], 48, 48)
        self.layer2 = self._make_layer(block, layers[1], 48, 96)
        self.layer3 = self._make_layer(block, layers[2], 96, 192)
        self.layer4 = self._make_layer(block, layers[3], 192, 384)

        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(384, 2)

    def forward(self, input):
        x = self.relu(self.bn1(self.conv1(input)))
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.maxpool(x)

        x = self.layer2(x)
        x = self.maxpool(x)

        x = self.layer3(x)
        x = self.maxpool(x)

        x = self.layer4(x)
        x = self.maxpool(x)

        x = self.avg_pool(x)
        x = self.flatten(x)
        x = self.fc(x)

        return x

    def _make_layer(self, block, layer, in_channels, out_channels):
        self.layer =layer
        layers = []
        self.out_channels = out_channels
        self.in_channels = in_channels
        identity_downsample = None

        self.expansion = 2

        
        if self.out_channels != self.in_channels:
            identity_downsample  = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1),
                nn.BatchNorm1d(out_channels)
            )
        layers.append(block(in_channels, out_channels, identity_downsample))
        self.in_channels = self.out_channels
            
        for i in range(layer-1):
            layers.append(block(self.in_channels, self.out_channels))

        return nn.Sequential(*layers)

In [12]:
layers = [3,4,6,3]
model = M34(block, 1, layers)

In [13]:
from torchsummary import summary
summary(model, (1,64000))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1            [-1, 48, 15961]           7,728
       BatchNorm1d-2            [-1, 48, 15961]              96
              ReLU-3            [-1, 48, 15961]               0
         MaxPool1d-4             [-1, 48, 3990]               0
            Conv1d-5             [-1, 48, 3990]           6,960
       BatchNorm1d-6             [-1, 48, 3990]              96
              ReLU-7             [-1, 48, 3990]               0
            Conv1d-8             [-1, 48, 3990]           6,960
       BatchNorm1d-9             [-1, 48, 3990]              96
             ReLU-10             [-1, 48, 3990]               0
            block-11             [-1, 48, 3990]               0
           Conv1d-12             [-1, 48, 3990]           6,960
      BatchNorm1d-13             [-1, 48, 3990]              96
             ReLU-14             [-1, 4